https://blog.csdn.net/qq_41718325/article/details/110092974

In [1]:
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer

class TreeNode(object):
    
    def __init__(self, model=None, C=None, left=None, right=None):
        self.model = model
        self.C = C
        self.left = left
        self.right = right

def trainLinear(linear, x, y):
    #使用sklearn库的最小二乘估计训练一个线性模型
    linear.fit(x, y)
    return linear

def binaryTrainSet(linear, x, y):
    #根据线性回归模型二分数据集
    #对样本x[i],其线性模型预测值若小于等于0,分到x0集合;若大于0,分到x1集合;相应的标签也划分的y0,y1集合
    x0 = []
    x1 = []
    y0 = []
    y1 = []
    p = linear.predict(x)
    for i in range(p.shape[0]):
        if p[i] <= 0:
            x0.append(x[i])
            y0.append(y[i])
        else:
            x1.append(x[i])
            y1.append(y[i])
    return np.array(x0), np.array(x1), np.array(y0), np.array(y1)

def score(linear, x, y):
    #计算线性模型linear的精度
    right = 0
    p = linear.predict(x)
    for i in range(p.shape[0]):
        if p[i]<=0 and y[i]==-1 or p[i]>0 and y[i]==1:
            right += 1
    return right / x.shape[0]
    
def treeGenerate(root, x, y, precision):
    #递归建造决策树
    root.model = LinearRegression()
    root.model = trainLinear(root.model, x, y)
    x0, x1, y0, y1 = binaryTrainSet(root.model, x, y)
    
    #构建当前结点左分支
    if len(x0)==0 or score(root.model, x0, y0)>= precision:
        #左分支训练集为空或当前结点的线性模型对左分支的训练样本精度达到了阈值要求(precision),将左分支构建为叶子节点
        root.left = TreeNode(C=-1)
    else:
        #左分支结点精度不够要求,还需进行划分
        root.left = TreeNode()
        treeGenerate(root.left, x0, y0, precision)
    
    #构建当前结点右分支
    if len(x1)==0 or score(root.model, x1, y1) >= precision:
        root.right = TreeNode(C=1)
    else:
        root.right = TreeNode()
        treeGenerate(root.right, x1, y1, precision)

def predict(root, xs):
    #使用以root为根结点的决策树预测样本s
    if root.C is not None:
        #root为叶子结点
        return root.C
    else:
        if root.model.predict(np.expand_dims(xs, axis=0)) <= 0:
            return predict(root.left, xs)
        else:
            return predict(root.right, xs)

def evaluate(root, x, y):
    #计算以root为根结点的决策树在数据集x上的精度
    right = 0
    for i in range(x.shape[0]):
        if predict(root, x[i]) == y[i]:
            right += 1
    return right / x.shape[0]

if __name__ == '__main__':
    #加载乳腺癌数据集
    cancer = load_breast_cancer()

    #参数random_state是指随机生成器,测试集占全部数据的33%
    X_train, X_test, y_train, y_test = train_test_split(cancer['data'],cancer['target'], test_size=0.33, random_state=42)
    
    #将y_train与y_test标签中的0全部改为-1
    y_train[y_train == 0] = -1
    y_test[y_test == 0] = -1

    #数据标准化
    X_train = preprocessing.scale(X_train)
    X_test = preprocessing.scale(X_test)
    
    #构建决策树
    root = TreeNode()
    #此处的阈值不能设的太大,由于数据本身就有一定客观存在的误差,无法做到100%精度,阈值设的太大容易爆栈
    treeGenerate(root, X_train, y_train, 0.96)
    
    #计算训练好的决策树在测试集上的精度
    scoreTrain = evaluate(root, X_train, y_train)
    scoreTest = evaluate(root, X_test, y_test)
    print('训练集精度为:', round(scoreTrain,4))
    print('测试集精度为:', round(scoreTest, 4))

训练集精度为: 0.979
测试集精度为: 0.9628
